In [19]:
# Title: Enchanted Forest Treasure Hunt - Python Implementation by:
#Group Members:
#1. Maryam Shah
#2.Laiba Shahid

import random
import heapq

# Define constants 
SAFE_GRID = 'O'
OBSTACLE = -1
START = 2
GOAL = 3
ENDPOINT = 5

# Define grid size
GRID_SIZE = 8

# Define rough probability for animal encounter
ANIMAL_ENCOUNTER_PROBABILITY = 0.8

# Define cost ranges
SAFE_COST_RANGE = (-4, -1)
OBSTACLE_COST_RANGE = (-4, -2)
ANIMAL_KILLED_COST_RANGE = (-4, -2)
ANIMAL_SURVIVED_COST_RANGE = (-4, -1)
ENDPOINT_COST = float('-inf')

# Define heuristic function
def manhattan_distance(current, goal):
    return abs(goal[0] - current[0]) + abs(goal[1] - current[1])

# function for random grid generation
def generate_random_grid():
    grid = [[SAFE_GRID for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
    grid[0][0] = START
    grid[GRID_SIZE-1][GRID_SIZE-1] = GOAL

    # Add obstacles
    for _ in range(10):  # Number of obstacles
        row = random.randint(0, GRID_SIZE-1)
        col = random.randint(0, GRID_SIZE-1)
        if grid[row][col] == SAFE_GRID:
            grid[row][col] = OBSTACLE

    # Add endpoints
    for _ in range(3):  # Number of endpoints
        row = random.randint(0, GRID_SIZE-1)
        col = random.randint(0, GRID_SIZE-2)
        if grid[row][col] == SAFE_GRID and grid[row][col+1] == SAFE_GRID:
            grid[row][col] = ENDPOINT
            grid[row][col+1] = ENDPOINT

    return grid

# function for rough animal encounter
def animal_encounter():
    return random.random() < ANIMAL_ENCOUNTER_PROBABILITY

# cost function for each grid element
def get_cost(grid_value):
    if grid_value == SAFE_GRID:
        return random.randint(*SAFE_COST_RANGE)
    elif grid_value == OBSTACLE:
        return random.randint(*OBSTACLE_COST_RANGE)
    elif grid_value == ENDPOINT:
        return ENDPOINT_COST

    # For animal encounter
    if animal_encounter():
        return random.randint(*ANIMAL_KILLED_COST_RANGE)
    else:
        return random.randint(*ANIMAL_SURVIVED_COST_RANGE)

# Uniform Cost Search (UCS)
def ucs(grid):
    start = (0, 0)
    goal = (GRID_SIZE-1, GRID_SIZE-1)
    visited = set()
    queue = [(0, start, [])]

    while queue:
        cost, current, path = heapq.heappop(queue)
        if current == goal:
            return path + [current], cost

        if current in visited:
            continue

        visited.add(current)
        row, col = current
        neighbors = [(row+1, col), (row-1, col), (row, col+1), (row, col-1)]
        for neighbor in neighbors:
            n_row, n_col = neighbor
            if 0 <= n_row < GRID_SIZE and 0 <= n_col < GRID_SIZE:
                if grid[n_row][n_col] != OBSTACLE and grid[n_row][n_col] != ENDPOINT:
                    heapq.heappush(queue, (cost + get_cost(grid[n_row][n_col]), neighbor, path + [current]))

    return None, float('inf')  # No path found

# function to display the grid
def display_grid(grid):
    print("Grid with Animals:")
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            print(f"({i},{j}): {grid[i][j]}", end="\t")
        print()

# function to display the UCS path
def display_path(path, total_cost, actions):
    print("\nUCS Path and Cost:")
    if path:
        print("Optimal Path Coordinates:")
        for coord in path:
            print(coord, end=" -> ")
        print("\nTotal Cost:", total_cost)
        print("Actions taken by animals along the path:")
        for i in range(len(path)-1):
            current = path[i]
            next_step = path[i+1]
            print(f"From {current} to {next_step}: {actions[(current, next_step)]}")
    else:
        print("No path found.")

# function to trace actions of animals along the path
def trace_animal_actions(grid, path):
    actions = {}
    for i in range(len(path)-1):
        current = path[i]
        next_step = path[i+1]
        if grid[next_step[0]][next_step[1]] == SAFE_GRID and animal_encounter():
            if next_step[0] > current[0]:
                actions[(current, next_step)] = "Animal encounter: Animal moved down." # animal moving down
            elif next_step[0] < current[0]:
                actions[(current, next_step)] = "Animal encounter: Animal moved up." # animal moving up 
            elif next_step[1] > current[1]:
                actions[(current, next_step)] = "Animal encounter: Animal moved right." # animal moving right
            elif next_step[1] < current[1]:
                actions[(current, next_step)] = "Animal encounter: Animal moved left." # animal moving left 
        else:
            actions[(current, next_step)] = "No animal encounter."
    return actions

# Main function
def main():
    grid = generate_random_grid()
    display_grid(grid)
    path, total_cost = ucs(grid)
    actions = trace_animal_actions(grid, path)
    display_path(path, total_cost, actions)

if __name__ == "__main__":
    main()
    



Grid with Animals:
(0,0): 2	(0,1): 5	(0,2): 5	(0,3): -1	(0,4): -1	(0,5): O	(0,6): O	(0,7): O	
(1,0): O	(1,1): O	(1,2): -1	(1,3): O	(1,4): O	(1,5): O	(1,6): O	(1,7): O	
(2,0): O	(2,1): O	(2,2): -1	(2,3): O	(2,4): O	(2,5): -1	(2,6): O	(2,7): O	
(3,0): -1	(3,1): O	(3,2): O	(3,3): -1	(3,4): O	(3,5): O	(3,6): O	(3,7): O	
(4,0): -1	(4,1): O	(4,2): O	(4,3): O	(4,4): O	(4,5): 5	(4,6): 5	(4,7): O	
(5,0): O	(5,1): O	(5,2): -1	(5,3): O	(5,4): O	(5,5): O	(5,6): O	(5,7): O	
(6,0): O	(6,1): O	(6,2): O	(6,3): O	(6,4): O	(6,5): O	(6,6): O	(6,7): O	
(7,0): O	(7,1): O	(7,2): O	(7,3): O	(7,4): O	(7,5): O	(7,6): -1	(7,7): 3	

UCS Path and Cost:
Optimal Path Coordinates:
(0, 0) -> (1, 0) -> (1, 1) -> (2, 1) -> (3, 1) -> (4, 1) -> (5, 1) -> (5, 0) -> (6, 0) -> (6, 1) -> (7, 1) -> (7, 2) -> (6, 2) -> (6, 3) -> (5, 3) -> (4, 3) -> (4, 4) -> (5, 4) -> (6, 4) -> (7, 4) -> (7, 5) -> (6, 5) -> (5, 5) -> (5, 6) -> (5, 7) -> (6, 7) -> (7, 7) -> 
Total Cost: -72
Actions taken by animals along the path:
From (0, 0) t

In [35]:
def test_ucs():
    try:
        # To ensure a path exists from start to goal
        while True:
            grid = generate_random_grid()
            path, cost = ucs(grid)
            if path is not None:
                break

        # Check if the path is valid
        assert path is not None, "No path found"
        assert len(path) > 0, "Path should not be empty"
        assert path[0] == (0, 0), "Path should start from (0, 0)"
        assert path[-1] == (GRID_SIZE-1, GRID_SIZE-1), f"Path should end at ({GRID_SIZE-1}, {GRID_SIZE-1})"

        # Check if the cost is reasonable
        assert cost >= -4 * GRID_SIZE * GRID_SIZE, "Cost should be greater than or equal to the minimum possible cost"
        assert cost <= -1, "Cost should be less than or equal to the maximum possible cost"

        print("UCS Test Passed!")
    except AssertionError as e:
        print(f"UCS Test Failed: {e}")

test_ucs()



UCS Test Passed!


In [13]:
# Define A* Search
def astar(grid):
    start = (0, 0)
    goal = (GRID_SIZE-1, GRID_SIZE-1)
    visited = set()
    queue = [(manhattan_distance(start, goal), 0, start, [])]

    while queue:
        _, cost, current, path = heapq.heappop(queue)
        if current == goal:
            return path + [current], cost

        if current in visited:
            continue

        visited.add(current)
        row, col = current
        neighbors = [(row+1, col), (row-1, col), (row, col+1), (row, col-1)]
        for neighbor in neighbors:
            n_row, n_col = neighbor
            if 0 <= n_row < GRID_SIZE and 0 <= n_col < GRID_SIZE:
                if grid[n_row][n_col] != OBSTACLE and grid[n_row][n_col] != ENDPOINT:
                    heapq.heappush(queue, (manhattan_distance(neighbor, goal) + cost + get_cost(grid[n_row][n_col]), cost + get_cost(grid[n_row][n_col]), neighbor, path + [current]))

    return None, float('inf')  # No path found.

# Main function for A* search
def astar_main():
    grid = generate_random_grid()
    display_grid(grid)
    path, total_cost = astar(grid)
    actions = trace_animal_actions(grid, path)
    display_path(path, total_cost, actions)

if __name__ == "__main__":
    astar_main()


Grid with Animals:
(0,0): 2	(0,1): O	(0,2): 5	(0,3): 5	(0,4): O	(0,5): O	(0,6): O	(0,7): O	
(1,0): O	(1,1): O	(1,2): O	(1,3): -1	(1,4): O	(1,5): O	(1,6): O	(1,7): O	
(2,0): -1	(2,1): O	(2,2): O	(2,3): O	(2,4): -1	(2,5): O	(2,6): O	(2,7): O	
(3,0): O	(3,1): O	(3,2): O	(3,3): O	(3,4): O	(3,5): O	(3,6): O	(3,7): O	
(4,0): O	(4,1): O	(4,2): O	(4,3): -1	(4,4): O	(4,5): O	(4,6): -1	(4,7): O	
(5,0): O	(5,1): O	(5,2): O	(5,3): O	(5,4): O	(5,5): O	(5,6): O	(5,7): O	
(6,0): -1	(6,1): O	(6,2): O	(6,3): O	(6,4): O	(6,5): O	(6,6): O	(6,7): O	
(7,0): O	(7,1): -1	(7,2): -1	(7,3): -1	(7,4): -1	(7,5): O	(7,6): O	(7,7): 3	

UCS Path and Cost:
Optimal Path Coordinates:
(0, 0) -> (0, 1) -> (1, 1) -> (1, 2) -> (2, 2) -> (3, 2) -> (3, 3) -> (3, 4) -> (4, 4) -> (4, 5) -> (5, 5) -> (5, 6) -> (5, 7) -> (6, 7) -> (7, 7) -> 
Total Cost: -29
Actions taken by animals along the path:
From (0, 0) to (0, 1): No animal encounter.
From (0, 1) to (1, 1): No animal encounter.
From (1, 1) to (1, 2): Animal encounter: Anim

In [34]:
def test_astar():
    try:
        # To ensure a path exists from start to goal
        while True:
            grid = generate_random_grid()
            path, cost = astar(grid)
            if path is not None:
                break

        # Check if the path is valid
        assert path is not None, "No path found"
        assert len(path) > 0, "Path should not be empty"
        assert path[0] == (0, 0), "Path should start from (0, 0)"
        assert path[-1] == (GRID_SIZE-1, GRID_SIZE-1), f"Path should end at ({GRID_SIZE-1}, {GRID_SIZE-1})"

        print("A* Test Passed!")
    except AssertionError as e:
        print(f"A* Test Failed: {e}")

test_astar()



A* Test Passed!
